# jupyter初始化过程中自动加载常用包的设置方法
- 找`startup`文件夹
用户主目录下找到`.ipython/profile_default/startup/`
如果没有startup文件夹则新建一个
- 创建`start.py`文件
在`startup`文件夹内创建`start.py`文件
- 在`start.py`文件内写入初始化时要`import`的包

In [1]:
# 切分训练集和测试集

In [1]:
def data_split(features,labels,rate=0.7):
    """训练集和测试集切分
    
    params features: 输入的特征张量
    params labels: 输入的标签张量
    params rate: 训练集占所有 数据的比例
    return Xtrain,Xtest,ytrain,ytest: 返回特征张量的训练集以及标签张量的训练集、测试集、测试集，
    
    """
    import random
    
    num_examples = len(features) # 总数据量
    indices = list(range(num_examples)) # 数据集行索引
    random.shuffle(indices) # 乱序调整
    num_train = int(num_examples * rate) # 训练集数量
    indices_train = torch.tensor(indices[:num_train]) # 在已经乱序的前indices中挑出前num_train数量的索引值
    indices_test = torch.tensor(indices[num_train:])
    Xtrain = features[indices_train] # 训练集特征
    ytrain = labels[indices_train] # 训练集标签
    Xtest = features[indices_test] # 测试集特征
    ytest = labels[indices_test] # 测试集标签
    
    return Xtrain,Xtest,ytrain,ytest

In [2]:
import torch
f = torch.arange(10)
f

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
l = torch.arange(1,11)
l

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [4]:
data_split(f,l)

(tensor([0, 7, 2, 4, 9, 3, 6]),
 tensor([1, 5, 8]),
 tensor([ 1,  8,  3,  5, 10,  4,  7]),
 tensor([2, 6, 9]))

In [9]:
# 导入相关的包

# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn,optim
from torch.nn import MSELoss # class
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.tensorboard import SummaryWriter

# 导入自定义模块
from torchLearning import *

# 导入以下包从而使得可以在 jupyter 中的 cell 输出多个结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
torch.manual_seed(420)

features,labels = tensorGenReg()

Xtrain,Xtest,ytrain,ytest = data_split(features,labels)

In [11]:
Xtrain.shape,Xtest.shape,ytrain.shape,ytest.shape

(torch.Size([700, 3]),
 torch.Size([300, 3]),
 torch.Size([700, 1]),
 torch.Size([300, 1]))

In [13]:
# 初始化核心参数
batch_size = 10
lr = 0.03
num_epochs = 5
w = torch.zeros(3,1,requires_grad=True)

# 参与训练的模型参数
net = linreg
loss = squared_loss

# 模型训练过程
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,Xtrain,ytrain):
        l = loss(net(X,w),y)
        l.backward()
        sgd(w,lr)

In [14]:
w

tensor([[ 2.0004],
        [-1.0010],
        [ 1.0008]], requires_grad=True)

In [15]:
squared_loss(torch.mm(Xtrain,w),ytrain)

tensor(0.0001, grad_fn=<DivBackward0>)

In [16]:
squared_loss(torch.mm(Xtest,w),ytest)

tensor(9.7553e-05, grad_fn=<DivBackward0>)

In [17]:
# pytorch中的切分数据不是复制切分数据存储，而是逻辑关系上（数据存储地址）

In [18]:
# Dataset 和 DataLoader的基本使用方法和数据切分

In [19]:
from torch.utils.data import random_split

In [20]:
t = torch.arange(12).reshape(4,3)
t

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

In [21]:
random_split(t,[2,2]) # 输入切分的每部分数据集数量
# params1: 切分的 数据tensor
# params2: [2,2] 切分为两份，分别两条数据 [6,4] 切分为两份，第一份6个，第二份4个
# random_split 函数其实生成了 生成器切分结果的 生成器，并不是切分数据后返回。这符合utils.data模块主要生成映射式和迭代对象的一般规定

In [22]:
train, test = random_split(t,[2,2])

In [23]:
for tr,te in random_split(t,[2,2]):
    print(tr,te)

tensor([ 9, 10, 11]) tensor([0, 1, 2])
tensor([3, 4, 5]) tensor([6, 7, 8])


In [26]:
from sklearn.datasets import load_breast_cancer as LBC # 导入乳腺癌数据
data = LBC()

In [29]:
data.data # 返回数据集的特征数组

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [31]:
data.target # 返回数据集的标签数组

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [32]:
len(data.data) # 返回数据集总个数

569

In [33]:
class LBCDataset(Dataset):
    def __init__(self,data): # 创建该类时需要输入sklearn导入的数据集
        self.features = data.data # features属性返回数据集特征
        self.labels = data.target # labels属性返回数据集标签
        self.lens = len(data.data) # lens属性返回数据集大小
        
    def __getitem__(self,index):
        # 调用该方法时需要输入index数值，方法最终返回index对应的特征和标签
        return self.features[index,:],self.labels[index]
    
    def __len__(self):
        # 调用该方法不需要输入额外参数，方法最终返回数据集大小
        return self.lens

In [34]:
data = LBC()
LBC_data = LBCDataset(data)

In [35]:
LBC_data.features

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [36]:
LBC_data.__getitem__(2) # 查看第3条数据

(array([1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e+01, 2.553e+01, 1.525e+02, 1.709e+03,
        1.444e-01, 4.245e-01, 4.504e-01, 2.430e-01, 3.613e-01, 8.758e-02]),
 0)

In [37]:
LBC_data.features[2]

array([1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
       1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
       4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
       2.250e-02, 4.571e-03, 2.357e+01, 2.553e+01, 1.525e+02, 1.709e+03,
       1.444e-01, 4.245e-01, 4.504e-01, 2.430e-01, 3.613e-01, 8.758e-02])

In [38]:
LBC_data.labels[2]

0

In [40]:
LBC_data[2] # 封装好的数据可以直接进行索引，并且能够返回实体结果

(array([1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e+01, 2.553e+01, 1.525e+02, 1.709e+03,
        1.444e-01, 4.245e-01, 4.504e-01, 2.430e-01, 3.613e-01, 8.758e-02]),
 0)

In [41]:
LBC_data[:] # 所有数据

(array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

In [42]:
# 使用random_split方法对其进行切分

In [44]:
num_train = int(LBC_data.lens * 0.7)
num_test = LBC_data.lens - num_train

In [45]:
num_train
num_test

398

171

In [47]:
LBC_train,LBC_test = random_split(LBC_data,[num_train,num_test])

# 此时切分的结果是一个映射式的对象，只有dataset 和 indices 两个属性，
# 其中dataset属性用于查看数据集对象，
# indices属性用于查看切分后数据集的每一条数据的index

In [48]:
LBC_train?

Type:        Subset
String form: <torch.utils.data.dataset.Subset object at 0x7fbe41b8fb80>
Length:      398
File:        ~/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataset.py
Docstring:  
Subset of a dataset at specified indices.

Arguments:
    dataset (Dataset): The whole Dataset
    indices (sequence): Indices in the whole set selected for subset


In [49]:
LBC_train.dataset

In [51]:
LBC_train.dataset == LBC_data

True

In [50]:
LBC_train.indices

[423,
 192,
 83,
 149,
 181,
 295,
 135,
 469,
 93,
 257,
 341,
 168,
 164,
 26,
 381,
 299,
 380,
 274,
 100,
 563,
 65,
 61,
 71,
 29,
 86,
 157,
 263,
 67,
 564,
 139,
 42,
 554,
 272,
 143,
 255,
 101,
 134,
 220,
 431,
 522,
 206,
 271,
 278,
 294,
 248,
 525,
 322,
 9,
 55,
 163,
 548,
 302,
 532,
 438,
 102,
 96,
 72,
 396,
 339,
 420,
 212,
 285,
 166,
 186,
 509,
 340,
 528,
 242,
 390,
 362,
 315,
 347,
 370,
 191,
 270,
 27,
 459,
 279,
 284,
 489,
 35,
 467,
 266,
 519,
 75,
 524,
 201,
 208,
 421,
 111,
 39,
 145,
 103,
 48,
 224,
 218,
 205,
 481,
 195,
 54,
 88,
 385,
 1,
 73,
 58,
 393,
 281,
 258,
 526,
 31,
 401,
 487,
 561,
 433,
 259,
 170,
 344,
 404,
 543,
 13,
 202,
 425,
 353,
 44,
 50,
 245,
 472,
 151,
 57,
 368,
 442,
 4,
 518,
 64,
 11,
 418,
 463,
 280,
 328,
 108,
 122,
 323,
 223,
 529,
 539,
 84,
 99,
 372,
 249,
 209,
 161,
 403,
 558,
 34,
 19,
 483,
 534,
 28,
 342,
 38,
 398,
 565,
 364,
 480,
 553,
 51,
 5,
 535,
 379,
 549,
 136,
 289,
 303,
 464,


In [52]:
for i in LBC_train:
    print(i)
    break

(array([1.366e+01, 1.913e+01, 8.946e+01, 5.753e+02, 9.057e-02, 1.147e-01,
       9.657e-02, 4.812e-02, 1.848e-01, 6.181e-02, 2.244e-01, 8.950e-01,
       1.804e+00, 1.936e+01, 3.980e-03, 2.809e-02, 3.669e-02, 1.274e-02,
       1.581e-02, 3.956e-03, 1.514e+01, 2.550e+01, 1.014e+02, 7.088e+02,
       1.147e-01, 3.167e-01, 3.660e-01, 1.407e-01, 2.744e-01, 8.839e-02]), 1)


In [86]:
# 完整建模评估过程

# 生成数据
features,labels = tensorGenReg()
features = features[:, :-1] # 删除最后全是1的列

# 创建一个针对手动创建数据的数据类
class GenData(Dataset):
    def __init__(self,features,lables): # 创建该类时需要输入的数据集
        self.features = features
        self.labels  = labels
        self.lens = len(features)
        
    def __getitem__(self,index):
        return self.features[index,:],self.labels[index]
    
    def __len__(self):
        return self.lens
    
# 实例化对象
data = GenData(features,labels)

# 切分数据集
num_train = int(data.lens * 0.7)
num_test = data.lens - num_train
data_train,data_test = random_split(data,[num_train,num_test])

# 加载数据
train_loader = DataLoader(data_train,batch_size=10,shuffle=True)
test_loader = DataLoader(data_test,batch_size=10,shuffle=False)

In [57]:
# 构建模型

# 初始化核心参数
batch_size = 10
num_epochs = 5
lr = 0.03

# Stage 1. 定义模型

class LR(nn.Module):
    # 定义模型点线结构
    def __init__(self,in_features=2,out_features=1):
        super(LR,self).__init__()
        self.linear = nn.Linear(in_features,out_features)
    
    # 定义模型正向传播规则   
    def forward(self,x):
        out = self.linear(x)
        
        return out
    
# 实例化模型
LR_model = LR()

# Stage 2. 定义损失函数
criterion = nn.MSELoss()

# Stage 3. 定义优化方法
optimizer = optim.SGD(LR_model.parameters(),lr = lr)

# Stage 4. 模型训练与测试
def fit(net,criterion,optimizer,batchData,epochs=3):
    for epoch in range(epochs):
        for X,y in batchData:
            yhat = net.forward(X)
            loss = criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [58]:
# 模型训练与测试

fit(net = LR_model,
   criterion = criterion,
   optimizer = optimizer,
   batchData = train_loader,
   epochs = num_epochs)

In [59]:
# 查看训练模型
LR_model

LR(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [60]:
# 查看模型参数
list(LR_model.parameters())

[Parameter containing:
 tensor([[ 2.0001, -1.0002]], requires_grad=True),
 Parameter containing:
 tensor([0.9997], requires_grad=True)]

In [61]:
# 查看模型在训练集上的表现

In [62]:
# 首先使用dataset和indices方法，还原训练数据集
data_train.indices # 返回训练集索引

[707,
 931,
 304,
 695,
 640,
 13,
 932,
 353,
 493,
 964,
 685,
 154,
 433,
 77,
 960,
 635,
 509,
 262,
 514,
 278,
 987,
 28,
 425,
 683,
 274,
 427,
 639,
 596,
 550,
 763,
 978,
 422,
 779,
 543,
 122,
 775,
 866,
 519,
 347,
 50,
 8,
 911,
 309,
 990,
 251,
 874,
 754,
 557,
 791,
 548,
 474,
 55,
 187,
 121,
 783,
 730,
 159,
 469,
 115,
 376,
 343,
 183,
 653,
 500,
 141,
 487,
 577,
 149,
 169,
 655,
 605,
 310,
 511,
 455,
 714,
 586,
 504,
 542,
 818,
 464,
 821,
 746,
 555,
 74,
 463,
 982,
 494,
 79,
 189,
 563,
 980,
 197,
 170,
 43,
 80,
 684,
 850,
 109,
 14,
 993,
 81,
 380,
 153,
 317,
 287,
 868,
 995,
 67,
 424,
 253,
 949,
 742,
 152,
 299,
 62,
 607,
 590,
 56,
 471,
 429,
 404,
 693,
 100,
 751,
 887,
 218,
 324,
 131,
 954,
 381,
 689,
 132,
 151,
 194,
 286,
 580,
 198,
 892,
 441,
 745,
 510,
 454,
 275,
 623,
 396,
 162,
 762,
 551,
 42,
 836,
 657,
 857,
 674,
 498,
 582,
 407,
 389,
 673,
 811,
 790,
 700,
 967,
 432,
 89,
 459,
 382,
 810,
 138,
 827,
 482

In [63]:
data[data_train.indices] # 返回训练集

(tensor([[-1.0566,  0.6108],
         [-0.0324, -1.0479],
         [-0.2432, -0.3873],
         ...,
         [-1.7935, -1.0075],
         [ 0.5288, -0.1329],
         [ 0.2610, -0.0919]]),
 tensor([[-1.7242e+00],
         [ 1.9906e+00],
         [ 8.9809e-01],
         [ 1.2759e+00],
         [-2.3948e+00],
         [-1.5561e+00],
         [-7.2017e-01],
         [ 4.5488e+00],
         [ 4.2594e+00],
         [ 8.1584e-01],
         [-9.2630e-01],
         [ 2.5102e+00],
         [ 4.6704e+00],
         [ 9.2945e-01],
         [ 1.7095e+00],
         [-4.7393e-01],
         [ 4.3947e+00],
         [ 2.4332e+00],
         [-2.1711e+00],
         [ 3.5524e+00],
         [-5.4729e-01],
         [ 1.3865e+00],
         [-2.7487e+00],
         [-9.4334e-01],
         [ 3.7239e+00],
         [ 5.8758e+00],
         [ 2.4333e+00],
         [ 3.1600e+00],
         [ 2.9291e+00],
         [ 1.0294e+00],
         [ 1.7596e+00],
         [-1.2694e+00],
         [ 2.8442e+00],
         [ 1.2511e

In [64]:
data[data_train.indices][0] # 返回训练集的特征

tensor([[-1.0566,  0.6108],
        [-0.0324, -1.0479],
        [-0.2432, -0.3873],
        ...,
        [-1.7935, -1.0075],
        [ 0.5288, -0.1329],
        [ 0.2610, -0.0919]])

In [68]:
data[data_test.indices][1]

tensor([[ 1.0077e+00],
        [ 1.6869e+00],
        [-6.6230e-02],
        [ 3.4079e+00],
        [ 2.5114e+00],
        [ 2.4704e+00],
        [ 2.4293e+00],
        [ 1.4161e+00],
        [ 4.0741e+00],
        [ 3.8438e+00],
        [-1.9990e+00],
        [ 2.3226e+00],
        [-2.6314e-01],
        [ 1.3193e-01],
        [ 1.2485e+00],
        [ 1.1407e+00],
        [ 1.2932e+00],
        [-2.2423e-01],
        [-7.4660e-01],
        [ 5.9922e-04],
        [-1.2848e-01],
        [ 1.1573e+00],
        [-1.1735e+00],
        [ 2.2792e+00],
        [ 2.4111e+00],
        [-1.4878e+00],
        [-1.3235e+00],
        [-2.6640e+00],
        [ 6.9863e-01],
        [-1.5456e+00],
        [ 7.3890e-01],
        [ 1.5608e-01],
        [ 4.4044e+00],
        [ 6.5633e-01],
        [ 2.5553e+00],
        [ 1.1644e+00],
        [-4.0400e-01],
        [ 3.4508e-02],
        [ 3.4358e+00],
        [ 1.6474e+00],
        [ 1.2788e+00],
        [-4.4499e-02],
        [-8.0633e-01],
        [ 1

In [89]:
data.lens

1000

In [66]:
# 计算训练集MSE
F.mse_loss(LR_model(data[data_train.indices][0]), data[data_train.indices][1])

tensor(9.5879e-05, grad_fn=<MseLossBackward>)

In [67]:
# 计算测试集MSE
F.mse_loss(LR_model(data[data_test.indices][0]), data[data_test.indices][1])

tensor(9.2361e-05, grad_fn=<MseLossBackward>)

## 实用数据补充

In [119]:
# 创建一个针对手动创建数据的数据类
class GenData(Dataset):
    def __init__(self,features,lables): # 创建该类时需要输入的数据集
        self.features = features
        self.labels  = labels
        self.lens = len(features)
        
    def __getitem__(self,index):
        return self.features[index,:],self.labels[index]
    
    def __len__(self):
        return self.lens
    
def split_loader(features,labels,batch_size=10,rate=0.7):
    """数据封装、切分和加载数据
    
    param features: 输入的特征
    param labele: 数据集标签张量
    param batch_size: 数据加载时的每一个小批数据量
    param rate: 训练集数据占比
    return : 加载好的训练集和测试集
    
    """
    data = GenData(features,labels)
    num_train = int(data.lens * 0.7)
    num_test = data.lens - num_train
    
    data_train,data_test = random_split(data,[num_train,num_test])
    train_loader = DataLoader(data_train,batch_size=batch_size,shuffle=True)
    test_loader = DataLoader(data_test,batch_size=batch_size,shuffle=False)
    
    return (train_loader,test_loader)

In [120]:
def fit(net,criterion,optimizer,batchData,epochs=3,cla=False):
    """模型训练函数
    param net: 待训练的模型
    param citerion: 损失函数
    param optimizer: 优化算法
    param batchData: 训练数据集
    param cla: 是否是分类问题
    param epochs: 遍历数据次数
    
    """
    
    for epoch in range(epochs):
        for X,y in batchData:
            if cla == True:
                y = y.flatten().long() # 如果是分类问题，要对y进行整数转化
            yhat = net.forward(X)
            loss = criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [121]:
def mse_cal(data_loader,net):
    """mse计算函数
    
    param data_Loader:加载好的数据
    param net:模型
    return :根据输入的数据，输出其MSE计算的结果
    
    """
    data = data_loader.dataset # 还原 Dataset类
    x = data[:][0] # 还原数据的特征
    y = data[:][1] # 还原数据的标签
    yhat = net(x)
    
    return F.mse_loss(yhat,y)

In [122]:
train_loader.dataset[:][0]

tensor([[-1.1511, -1.5656],
        [-1.3589,  0.3561],
        [ 0.3944, -1.5511],
        ...,
        [ 1.7895,  1.0660],
        [-0.8417,  1.8088],
        [-0.4877,  0.3996]])

In [123]:
torch.manual_seed(420)

# 实例化模型
LR_model = LR()

criterion = nn.MSELoss()

optimizer = optim.SGD(LR_model.parameters(),lr=0.03)

fit(net = LR_model,
    criterion = criterion,
    optimizer = optimizer,
    batchData = train_loader,
    epochs = 3
)

In [124]:
mse_cal(train_loader,LR_model) # 计算训练误差

tensor(0.0001, grad_fn=<MseLossBackward>)

In [125]:
mse_cal(test_loader,LR_model) # 计算测试误差

tensor(9.4898e-05, grad_fn=<MseLossBackward>)

In [126]:
# 准确率计算函数
def accuracy_cal(data_loader,net):
    """准确率
    param data_loader: 加载好的数据
    param net: 数据
    return : 根据输入的数据，输出其准确率计算结果
    
    """
    data = data_loader.dataset # 还原 Dataset 类
    X = data[:][0] # 还原数据的特征
    y = data[:][1] # 还原数据的标签
    zhat = net(X) # 默认是分类问题，并且输出结果是未经softmax转化的结果
    soft_z = F.softmax(zhat,1) # 进行softmax转化
    acc_bool = torch.argmax(soft_z,1).flatten() == y.flatten() # 每条数据最大值结果所属的类别与标签是否一致 1 列
    acc = torch.mean(acc_bool.float())
    
    return acc

In [127]:
t = torch.arange(9).reshape(3,3).float()
t

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [128]:
F.softmax(t,1)

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])

In [129]:
torch.manual_seed(420)

features,labels = tensorGenCla()

train_loader,test_loader = split_loader(features,labels)

In [131]:
class softmaxR(nn.Module):
    def __init__(self,in_features=2,out_features=3,bias=False):
        super(softmaxR,self).__init__()
        self.linear = nn.Linear(in_features,out_features)
        
    def forward(self,x):
        out = self.linear(x)
        
        return out

# 实例化模型
softmax_model = softmaxR()

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(softmax_model.parameters(),lr=lr)

fit(net = softmax_model,
   criterion = criterion,
   optimizer = optimizer,
   batchData = train_loader,
   epochs= num_epochs,
   cla = True)

In [133]:
accuracy_cal(train_loader,softmax_model)

tensor(0.8790)

In [134]:
accuracy_cal(test_loader,softmax_model)

tensor(0.8711)